___
<h1> Machine Learning </h1>
<h2> M. Sc. in Electrical and Computer Engineering </h2>
<h3> Instituto Superior de Engenharia / Universidade do Algarve </h3>

[MEEC](https://ise.ualg.pt/en/curso/1477) / [ISE](https://ise.ualg.pt) / [UAlg](https://www.ualg.pt)

Pedro J. S. Cardoso (pcardoso@ualg.pt)
___

_Note: running this notebook will, probably, require some hours._ 

# Support vector machines (SVMs) 

Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:
* Effective in high dimensional spaces.
* Still effective in cases where number of dimensions is greater than the number of samples.
* Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
* Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

See https://scikit-learn.org/stable/modules/svm.html for an explanation of the module and https://scikit-learn.org/stable/modules/svm.html#svm-mathematical-formulation for a the mathematical formulation.



## Classification

Let us start with a simple example of classification using SVM. We will use the iris dataset and, as usual, we will split the dataset into training and test sets.

In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris.data,
                                                    iris.target, 
                                                    random_state=10)

Now, let us train a SVM classifier using the training set and test it using the test set.

In [ ]:
svm = SVC(C=.1, 
          kernel='poly', 
          degree=4).fit(X_train, y_train)

score = svm.score(X_test, y_test)
score

Changing C and kernel parameters, we can get better results.

In [ ]:
svm = SVC(
    C=.01, 
    kernel='poly', 
    degree=4).fit(X_train, y_train)

score = svm.score(X_test, y_test)
print(score)
print('"1.0!! Pure luke"!! try with other random state value (train_test_split)!')

See also https://scikit-learn.org/stable/auto_examples/svm/plot_iris_svc.html#sphx-glr-auto-examples-svm-plot-iris-svc-py

## Regression

Next we present a few examples of regression using SVM. 

Let us consider the Seoul Bike Sharing Demand dataset. The dataset contains the hourly count of rental bikes between years 2017 and 2018 in Seoul, Korea with the corresponding weather and seasonal information. The dataset can be downloaded from https://archive.ics.uci.edu/dataset/560/seoul+bike+sharing+demand but we have already downloaded it and saved it in the data folder.

Let us start by loading the dataset into a pandas dataframe. 

In [ ]:
import pandas as pd
df = pd.read_csv('./data/SeoulBikeData.csv')
df.head()

By calling the dataframe's info method, we can see that there are no missing values but there are some categorical columns.

In [ ]:
df.info()

The categorical columns need to be converted into, for example, dummy variables. 

A dummy variable is a numerical variable used in regression analysis to represent subgroups of the sample in your study. In research design, a dummy variable is often used to distinguish different treatment groups. for example the season column has four categories: Spring, Summer, Autumn, and Winter. We can convert this column into four columns, one for each category, and use 0 or 1 to indicate if the sample belongs to that category or not. To achieve this, we can use the pandas get_dummies method.

In [ ]:
df = pd.get_dummies(df, columns=['Seasons', 'Holiday', 'Functioning Day'], drop_first=True)
df

We can split this column into two columns: month and day, and day of week. To achieve this, we can use the pandas to_datetime method as follows:

In [ ]:
# make sure the date column is in datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# create new columns for month, day, and day of week
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['day_of_week'] = df['Date'].dt.day_of_week

# drop the original date column
df.drop('Date', axis=1, inplace=True)

Let us now recheck the dataframe's info method.

In [ ]:
df.info()

Since the target variable is the Rented Bike Count, we can split the dataframe into two dataframes: one with the target variable and another with the remaining variables.

In [ ]:
X = df.drop('Rented Bike Count', axis=1)
y = df['Rented Bike Count']

Following the usual procedure, we can split the dataset into training and test sets. Shuffling the dataset is important to avoid any ordering bias.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    shuffle=True,
                                                    random_state=42,
                                                    test_size=0.2)

To train a SVM regressor, we can use the SVR class from the sklearn.svm module. Furthermore, we can use the GridSearchCV class to perform a grid search to find the best parameters for the SVR model.

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

def create_model_with_GSCV(X_train, y_train):
    grid_search_parameters = [
        {'kernel': ['linear'], 'C': [10**i for i in range(-3, 4)]},
        {'kernel': ['rbf'], 'C': [10**i for i in range(-3, 4)], 'gamma': [10**i for i in range(-3, 4)]},
        {'kernel': ['poly'], 'C': [10**i for i in range(-3, 4)], 'degree': [2, 3]}
    ]
    
    # for a faster search while coding, use the following parameters (if you have time, try the above parameters)
    # you can also change the number of folds (cv) to 3
    # grid_search_parameters = [
    #     {'kernel': ['linear'], 'C': [0.1]},
    #     {'kernel': ['rbf'], 'C': [0.1], 'gamma': [0.1]},
    #     {'kernel': ['poly'], 'C': [0.1], 'degree': [0.1]}
    # ]
    
    # create the model
    svr = SVR()
    
    # create grid search and fit it to the training data
    gs_model = GridSearchCV(estimator=svr,
                            param_grid=grid_search_parameters,
                            cv=5,
                            n_jobs=-1,
                            verbose=1).fit(X_train, y_train)
    return gs_model

In [ ]:
gdcv_model = create_model_with_GSCV(X_train, y_train)

The best parameters and score can be obtained as follows:

In [ ]:
gdcv_model.best_params_

In [ ]:
gdcv_model.best_score_ 

Note that refit is by default=True, which means that the GridSearchCV will refit an estimator using the best found parameters on the whole dataset. And the best estimator can be obtained as follows:

In [ ]:
model = gdcv_model.best_estimator_

Over the test set, we can obtain the score as follows, which somehow indicates how well the model generalizes.

In [ ]:
model.score(X_test, y_test)

We can make predictions over the test set as follows and compare the predicted values with the actual values, by plotting them. On a prefect regression, the points would be on the diagonal.

In [ ]:
import matplotlib.pyplot as plt

# make predictions over the test set
pred = model.predict(X_test)

# plot pred vs actual
plt.figure(figsize=(10,10))
plt.plot(y_test.values, pred, c='g', marker='o', linestyle='None')
plt.plot([0,3500], [0, 3500], c='r')
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Actual vs Predicted")


# Feature engineering: SVM on a normalized & extended Seoul Bike Dataset

## Normalization
Some methods have difficulties to deal with data that are not normalized. Let us see how the Seoul Bike Data is distributed

In [ ]:
df.describe()

|Box plots also help with visualization of the distribution

In [ ]:
X_train.plot(kind='box', figsize=(20,5))

### Standardization
One usual solution is to normalize the distribution by subtracting the mean and dividing by the standard deviation, 

$$X'_{ij} = \frac{X_{ij}-\mu_j}{\sigma_j}$$

where X_{ij} is the observation $i$ for the feature $j$, $\mu_j$ is the mean and $\sigma_j$ is the standard deviation.

This can be done by coding or simply using sklearn 

In [ ]:
from sklearn.preprocessing import StandardScaler

# set and fit the scaler
standard_scaler = StandardScaler().fit(X_train)

X_train_normalized = pd.DataFrame(standard_scaler.transform(X_train), columns = X_train.columns)
X_train_normalized.plot(kind='box', figsize=(20,5))

So, let us create a model but now using the standarderized data

In [ ]:
gdcv_normalized_model = create_model_with_GSCV(X_train_normalized, y_train)

The best parameters and score can be obtained as follows:

In [ ]:
gdcv_normalized_model.best_params_

The best score is

In [ ]:
gdcv_normalized_model.best_score_

Note that refit is by default=True, which means that the GridSearchCV will refit an estimator using the best found parameters on the whole dataset. And the best estimator can be obtained as follows:

In [ ]:
model_normalized = gdcv_normalized_model.best_estimator_

Over the test set, we can obtain the score as follows, which somehow indicates how well the model generalizes.

In [ ]:
X_test_normalized = pd.DataFrame(standard_scaler.transform(X_test), columns = X_train.columns) 
pred_normalized = model_normalized.predict(X_test_normalized)

We can make predictions over the test set as follows and compare the predicted values with the actual values, by plotting them. On a prefect regression, the points would be on the diagonal.

In [ ]:
# plot pred vs actual
plt.plot( y_test.values, pred_normalized, c='g', marker='o', linestyle='None')

# plot the diagonal
plt.plot([0,3500], [0, 3500], c='r')

# set labels
plt.ylabel('Predicted')
plt.xlabel('Actual')
plt.title('Actual vs Predicted')

### MinMaxScaler

Another usual solution is to normalize the distribution by subtracting the minimum and dividing by the difference between the maximum and the minimum,

$$ X'_{ij} = \frac{X_{ij}-\min_j}{\max_j-\min_j}$$

where X_{ij} is the observation $i$ for the feature $j$, $\min_j$ is the minimum and $\max_j$ is the maximum. Returned values are in the range [0, 1].

This can be done by coding or simply using sklearn

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# set and fit the scaler
minmax_scaler = MinMaxScaler().fit(X_train)

X_train_minmax = pd.DataFrame(minmax_scaler.transform(X_train), columns = X_train.columns)
X_train_minmax.plot(kind='box', figsize=(20,5))

So, let us create a model but now using the standarderized data 

In [ ]:
gdcv_minmax_model = create_model_with_GSCV(X_train_minmax, y_train)

The best parameters and score can be obtained as follows:

In [ ]:
gdcv_minmax_model.best_params_

The best score is

In [ ]:
gdcv_minmax_model.best_score_

And, to compare the predictions with the actual values, by plotting them. On a prefect regression, the points would be on the diagonal.

In [ ]:
# make predictions over the test set
X_test_minmax = pd.DataFrame(minmax_scaler.transform(X_test), columns = X_train.columns)
pred_minmax = gdcv_minmax_model.best_estimator_.predict(X_test_minmax)

# plot pred vs actual
plt.plot(y_test.values, pred_minmax, c='g', marker='o', linestyle='None')
plt.plot([0,3500], [0, 3500], c='r')
plt.ylabel('Predicted')
plt.xlabel('Actual')
plt.title('Actual vs Predicted')

## Polynomial features

Other approach is to create polynomial features. In this case, if the original set of feature is $(x_1, x_2, x_n)$ then the polynomial features with degree 2 are $(1, x_1, x_2, x_n, x_1^2, x_1x_2, x_1x_n, x_2^2, x_2x_n, ...,  x_n^2 \ldots)$.

This can be done by coding or simply using sklearn

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# set and fit the scaler
poly = PolynomialFeatures(degree=2, include_bias=False).fit(X_train)

X_train_poly = pd.DataFrame(poly.transform(X_train), columns = poly.get_feature_names_out(X_train.columns))
X_test_poly = pd.DataFrame(poly.transform(X_train), columns = poly.get_feature_names_out(X_train.columns))

Train a model using the polynomial features

In [ ]:
gscv_poly_model = create_model_with_GSCV(X_train_poly, y_train)

The best parameters and score can be obtained as follows:

In [ ]:
gscv_poly_model.best_params_

The best score is

In [ ]:
gs_score = gscv_poly_model.score

And, to compare the predictions with the actual values, by plotting them. On a prefect regression, the points would be on the diagonal.

In [ ]:
pred_poly = gscv_poly_model.best_estimator_.predict(X_test_poly)

# plot pred vs actual
plt.plot(y_test.values, pred_poly, c='g', marker='o', linestyle='None')
plt.plot([0,3500], [0, 3500], c='r')

plt.ylabel('Predicted')
plt.xlabel('Actual')
plt.title('Actual vs Predicted')


## Normalization + Polynomial features

Now, let us combine both normalization and polynomial features

In [ ]:
# set and fit the scaler
standard_scaler = StandardScaler().fit(X_train)

# normalize the data
X_train_normalized = pd.DataFrame(standard_scaler.transform(X_train), columns = X_train.columns)
X_test_normalized = pd.DataFrame(standard_scaler.transform(X_test), columns = X_train.columns)

# set the polynomial features
poly = PolynomialFeatures(degree=2, include_bias=True).fit(X_train_normalized)

# create the polynomial features
X_train_normalized_poly = pd.DataFrame(poly.transform(X_train_normalized), columns = poly.get_feature_names_out(X_train_normalized.columns))
X_test_normalized_poly = pd.DataFrame(poly.transform(X_test_normalized), columns = poly.get_feature_names_out(X_test_normalized.columns))

# train the model
gscv_normalized_poly_model = create_model_with_GSCV(X_train_normalized_poly, y_train)

The best parameters and score can be obtained as follows:

In [ ]:
# get the best score
print(f'Best score: {gscv_normalized_poly_model.best_score_}')

# get the best parameters
print(f'Best parameters: {gscv_normalized_poly_model.best_params_}')

And, to compare the predictions with the actual values, by plotting them. On a prefect regression, the points would be on the diagonal.

In [ ]:
pred_normalized_poly = gscv_normalized_poly_model.best_estimator_.predict(X_test_normalized_poly)

# plot pred vs actual
plt.plot(y_test.values, pred_normalized_poly, c='g', marker='o', linestyle='None')
plt.plot([0,3500], [0, 3500], c='r')

plt.ylabel('Predicted')
plt.xlabel('Actual')
plt.title('Actual vs Predicted')
